![FREYA Logo](https://github.com/datacite/pidgraph-notebooks-python/blob/master/images/freya_200x121.png?raw=true) | [FREYA](https://www.project-freya.eu/en) | WP2 [User Story 7]( https://www.pidforum.org/t/pid-graph-graphql-example-second-degree-citations/939): As a data center, I want to see the citations of publications that use my repository for the underlying data, so that I can demonstrate the impact of our repository.
:------------- | :------------- | :-------------

It is important for repositories of scientific data to monitor and report on the impact of the data they store. One useful proxy of that impact are secondary citations, i.e. citations of publications which use the deposited data. This notebook focuses on visualisation of these citations by means of a force-directed graph.<p />
This notebook uses the [DataCite GraphQL API](https://api.datacite.org/graphql) to retrieve the citations of the following different datasets: 
- [Effects of varying food-availability on ecology and distribution of smallest benthic organisms in sediments of the arctic Fram Strait during POLARSTERN cruise ARK-XV/2, supplement to: Schewe, Ingo; Soltwedel, Thomas (2003): Benthic response to ice-edge-induced particle flux in the Arctic Ocean. Polar Biology, 26(9), 610-620](https://doi.org/10.1594/pangaea.314690);
- [Data from: Towards a worldwide wood economics spectrum](https://doi.org/10.5061/dryad.234); and
- [rmca-albertine-rift-cichlids](https://doi.org/10.15468/n6ftyd).

**Goal**: By the end of this notebook, for a given list of datasets, you should be able to display:
- Total citation count for each retrieved dataset;
- An interactive force-directed graph of the datasets and their citations, in which:
 - Each node at the centre of each radial shape corresponds to a dataset;
 - Vertices correspond to citations (note that some citations may be shared by more than one dataset);
 - The **more edges** come out of a (dataset or citation) node, the closer to **red** its colour becomes;
 - The width of each edge reflects the number of citations (second-degree) of the dataset citation (first-degree) represented by the vertex, e.g. <br><br><img src="example_plot.png" width="500" height="500" />"


## Install libraries and prepare GraphQL client

In [5]:
%%capture
# Install required Python packages
!pip install gql requests pyvis

In [6]:
# Prepare the GraphQL client
import requests
from IPython.display import display, Markdown
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

_transport = RequestsHTTPTransport(
    url='https://api.datacite.org/graphql',
    use_json=True,
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

## Define and run GraphQL query
Define the GraphQL query to find all publications including co-authors for [Dr Sarah Teichmann](https://orcid.org/0000-0002-6294-6366):

In [7]:
# Generate the GraphQL query to retrieve up to 100 researchers matching query "John and Smith"
query_params = {
    "ids" : ["10.5061/dryad.234","10.15468/n6ftyd","10.1594/pangaea.314690"]
}

query = gql("""query getDatasetCitations($ids: [String!]) {
  datasets(ids: $ids) {
    nodes {
      id
      titles {
        title
      }
      citationCount
      citations {
        nodes {
          id
          publisher
          titles {
            title
          }
          citationCount
        }
      }
    }
  }
}


""")

Run the above query via the GraphQL client

In [8]:
import json
data = client.execute(query, variable_values=json.dumps(query_params))

## Display total number of citations per dataset

In [11]:
# Get total citation counts for each dataset in the query
datasets = data['datasets']
tableBody=""
for dataset in datasets['nodes']:
    id = dataset['id']
    titles = []
    for title in dataset['titles']:
        titles.append(title['title'])
    citationCount = dataset['citationCount']
    tableBody += "[%s](%s) | **%s**\n" % (', '.join(titles), id, citationCount)
if tableBody:
    display(Markdown("| Dataset | Citation Count|\n|---|---|\n%s" % tableBody))  

| Dataset | Citation Count|
|---|---|
[Effects of varying food-availability on ecology and distribution of smallest benthic organisms in sediments of the arctic Fram Strait during POLARSTERN cruise ARK-XV/2, supplement to: Schewe, Ingo; Soltwedel, Thomas (2003): Benthic response to ice-edge-induced particle flux in the Arctic Ocean. Polar Biology, 26(9), 610-620](https://doi.org/10.1594/pangaea.314690) | **164**
[Data from: Towards a worldwide wood economics spectrum](https://doi.org/10.5061/dryad.234) | **4**
[rmca-albertine-rift-cichlids](https://doi.org/10.15468/n6ftyd) | **81**


## Plot a force-directed graph connecting datasets to their publications and citations of those publications
Plot an interactive force-directed graph of connecting the datasets to their citations (first-degree) and the citations of those citations (second-degree).
 - Each node at the centre of each radial shape corresponds to a dataset;
 - Vertices correspond to citations (note that some citations may be shared by more than one dataset);
 - The **more edges** come out of a (dataset or citation) node, the closer to **red** its colour becomes;
 - The width of each edge reflects the number of citations (second-degree) of the dataset citation (first-degree) represented by the vertex.

In [32]:
from pyvis.network import Network
import pandas as pd
from IPython.display import IFrame

# Colour swatch for the network nodes
colours = ["#00ffff", "#8000ff", "#bf00ff", "#ff00ff", "#ff00bf", "#ff0080", "#ff0040", "#ff0000"]

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
got_net.options.edges.inherit_colors(False)

# set the physics layout of the network
got_net.barnes_hut()

# ------------------------------
# Initialise intermediate data structure to store: (src, trg) -> citation count of the target, where:
# src - dataset or citation; trg - citation
srcTrg2Count = {}
# Initialise intermediate data structure to store: src --> Set of connected trg's
# Note that the number of connected trgs will determine the colour of each src
src2OtherTrgs = {}

datasets = data['datasets']

# Populate srcTrg2Count
allNodes = set()
for node in datasets['nodes']:
    nodeSet = set()
    datasetDOI = "/".join(node['id'].split("/")[3:])
    nodeSet.add(datasetDOI)
    for citation in node['citations']['nodes']:
        citationDOI = "/".join(citation['id'].split("/")[3:])
        # We're adding 1 below to make edges representing 0 citations of the target appear in the force-directed graph   
        citationCount = citation['citationCount'] + 1
        nodeSet.add(citationDOI)
        if datasetDOI not in src2OtherTrgs:
            src2OtherTrgs[datasetDOI] = set()
        src2OtherTrgs[datasetDOI].add(citationDOI)
        if citationDOI not in src2OtherTrgs:
            src2OtherTrgs[citationDOI] = set()   
        src2OtherTrgs[citationDOI].add(datasetDOI)        
        srcTrg2Count[(datasetDOI, citationDOI)] = citationCount     
    nodes = sorted(list(nodeSet))
    allNodes.update(nodes)

# Populate data structures needed for the graph
sources, targets, weights = [], [], []
for tuple in srcTrg2Count:
    if srcTrg2Count[tuple] > 0:
        sources.append(tuple[0])
        targets.append(tuple[1])
        weights.append(srcTrg2Count[tuple])

edge_data = zip(sources, targets, weights)

# Parameters used to determine node colours - the more edges come out of the node, the closer to red its colour becomes
max_len = len(allNodes)
num_colours = len(colours)
div = max_len/num_colours/3

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    colour_index = min(int(len(src2OtherTrgs[src])/div), num_colours - 1)
    got_net.add_node(src, src, title=src, color = colours[colour_index])   
    colour_index = min(int(len(src2OtherTrgs[dst])/div), num_colours - 1)
    got_net.add_node(dst, dst, title=dst, color = colours[colour_index])
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

got_net.show("out.html")
display(Markdown("N.B. Click on the plot, then use down/up mouse scroll to zoom in/out respectively.<br>When zoomed in, you will notice the DOI label against each node.<br>Click on a node to see the list of 'neighbour' citations."))
IFrame(src="./out.html", width=1000, height=800)

N.B. Click on the plot, then use down/up mouse scroll to zoom in/out respectively.<br>When zoomed in, you will notice the DOI label against each node.<br>Click on a node to see the list of 'neighbour' citations.